In [9]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from skimage import io
from skimage import img_as_float32
from scipy.signal import find_peaks
from skimage import transform

In [3]:
# Run only once or if the plot doesnt appear in a new window
%matplotlib qt


In [4]:
# Image Directory Location
IMG_DIRECTORY = r"D:\Scripts\MicroChip\images"
SAVE_TO_DIRECTORY = "" 
# File save name will be the img_dir folder name

# Channel parameters (in pixels)
WIN_LEN = 10 # How long the detection window length (vertical) should be
WALL_BUFFER = 2 # How many pixels from the wall to ignore for signal calculations

# Filtering Parameters
CUTOFF = 0.75
ORDER = 2


In [7]:
# Adjust the angle of the detection window (INTERACTIVE)
"""
Left click to adjust the height of the top left side of the detection window. 
Righ click to adjust the height of the top right side of the detection window. 

This will get the angle we need to rotate the image by so that the channels
are exactly straight up and down. 


"""
# Get the Image files prepped in a list
image_files = [ f for f in glob.glob(IMG_DIRECTORY+ '\\*.tiff')]


total_img = []
for i in image_files:
    img = img_as_float32(io.imread(i))
    if total_img ==[]:
        total_img = img
    else:
        total_img += img

window_length = WIN_LEN # pixels
wall_buffer = WALL_BUFFER #  pixels
fig, ax = plt.subplots(1)
#ax = axes[0]
ax.imshow(total_img)
line1, = ax.plot((0,total_img.shape[1]), [total_img.shape[0]/2]*2)
line2, = ax.plot((0, total_img.shape[1]), [total_img.shape[0]/2+window_length]*2)
def onclick(event):    
    prev_data = line1.get_ydata()
    if event.button == 1:
        prev_data[0]=event.ydata
    elif event.button == 3:
        prev_data[1]=event.ydata
    line1.set_ydata(prev_data)
    prev_data = np.add(prev_data, window_length)
    line2.set_ydata(prev_data)
    fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick)


D:\Software\MiniConda\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [10]:
# Once the Image rotation has been determined, get the Signal
def get_zoom_region(img):
    img = transform.rotate(img, theta, False, clip=True, preserve_range=True)
    img[img==0]=np.nan
    img = img[:,~np.any(np.isnan(img[int(top[0]):int(top[0])+window_length]), axis=0)]

    return img



def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    """
    numpy = np
    if x.ndim != 1:
        raise ValueError( "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError ( "Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError( "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y[int(window_len/2-1):-int(window_len/2)]

# Average all data between the two lines
top = line1.get_ydata()
bottom = line2.get_ydata()

# Rotate the image and retrive jus the zoomed portion
slope = (top[1]-top[0])/total_img.shape[1]
theta = np.arcsin((top[1]-top[0])/total_img.shape[1]) 
theta = theta * 180 / np.pi
img = get_zoom_region(total_img)

# Calculate where the walls of the channel are located
x = np.sum(img[int(top[0]):int(top[0])+window_length, :], axis=0)
dx = np.diff(smooth(x, window='bartlett') )
std = np.std(dx)

peaks, properties = find_peaks(-dx, prominence=( std))
peaks += 1
properties["prominences"].max()


# Deterimine if we are odd or even (wall or channel is on the edge)
if np.median(np.diff(peaks)[::2]) > np.median(np.diff(peaks)[1::2]):
    start_idx = 0
    end_idx = 1
else:
    start_idx = 1
    end_idx = 2


# Iterate through all the images and retrieve the data using the zoomed region and the wall locations
data = []
for img_file in image_files:
    row = []
    img = io.imread(img_file)
    img = get_zoom_region(img)
    x = np.sum(img[int(top[0]):int(top[0])+window_length, :], axis=0)
    for start, stop in zip(peaks[start_idx::2], peaks[end_idx::2]):
        row.append(np.sum(x[start+wall_buffer:stop-wall_buffer]))
    data.append(row)
    
data = np.asarray(data)


# Get the Time Data
from datetime import datetime
def get_time(file_name):
    str_time = file_name.split('\\')[-1].strip('.tiff').strip('img').strip()
    start_time = datetime.strptime(str_time, "%y-%m-%d %H-%M-%S %f")
    return start_time

time = []
start_time = get_time(image_files[0])
for image_file in image_files:
    current_time = get_time(image_file)
    dt = current_time-start_time
    time.append(dt.total_seconds())
    


In [11]:
# Filter the signal data 
# Filtering Functions
from scipy import signal
def butter_lowpass_filter(data, **kwargs):
    """
    Apply a butterworth lowpass filter to the data set.
    Keyword arguments will be used in a dictionary of filter settings:
    'cutoff' - frequency cutoff, should be < 1/2 the sampling frequency (fs)
    'fs' - sampling frequncy, the frequency measurments are recorded at
    'order' - order of the butter filter
    'padlen' - how much to pad the beginning and end of the array (see scipy.signal.butter)
    'padtype' - how to pad the ends of the array (see scipy.signal.butter)

    :param data: 1D equally spaced array
    :param kwargs: dict,
    :return:
    """

    def butter_lowpass(cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    settings = {'cutoff': 3, 'fs': 10, 'order': 5, 'padlen': 24, 'padtype': 'constant'}
    settings.update(kwargs)

    b, a = butter_lowpass(settings['cutoff'], settings['fs'], order=settings['order'])
    y = signal.filtfilt(b, a, data, padlen=settings['padlen'], padtype=settings['padtype'])

    return y

average_sampling_freq = 1/np.mean(np.diff(time))
filt_data = data.copy()
for i in range(data.shape[1]):
    y = data[:,i]
    filt_data[:,i] = butter_lowpass_filter(y, fs=average_sampling_freq, cutoff=0.7, order=2)
    

In [12]:
# Plot the Fitlered Data 

fig, ax = plt.subplots(1)
lines = ax.plot(filt_data)
for line in lines:
    line.set_xdata(time)
ax.set_xlim((0, max(time)))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Summed Intensity (au)')

Text(0, 0.5, 'Summed Intensity (au)')

In [17]:
# Save the data as CSV files
def out_data(fout, data, time):
    fout.write('Time')
    for i in range(data.shape[1]):
        fout.write(f',Chnl_{i}')
    fout.write('\n')
    for i in range(len(time)):
        fout.write(f'{time[i]}')
        for j in data[i,:]:
            fout.write(f',{j}')
        fout.write('\n')
        
file_name = IMG_DIRECTORY.split('\\')[-1]
with open('Raw_'+file_name+'.csv', 'w') as fout:
    out_data(fout, data, time)
with open('Filtered_'+file_name+'.csv', 'w') as fout:
    out_data(fout, filt_data, time)

        
    